# Rossmann

Rossmann is a chain of German grocery stores. There was a past kaggle competition
to predict the sales at particular stores on particular days. A few years of sales 
data was given, and then the competition participants were asked to predict sales 
over the following 2 weeks. 

The lesson 3 fastai notebook trains a structured data model to make sales prediction
following the approach of the third place winner in the kaggle competition. First, significant
pre-processing of the data is done to get "the right" continuous and categorical 
variables, and then a structured data model is trained. 

In this notebook I use the same pre-processing steps copied directly from the fastai
notebook, but then train my own structured data model once the appropriate variables have 
been abstracted. 

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# My Imports
import StructuredData as SD
import General as Gen

# Fastai Imports (these contain pytorch imports)
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

# Standard Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

PATH='data/rossmann/'

### Part (1) - Loading in data and preprocessing (Copied directly from Lesson 3 Fastai Notebook)

## Create datasets

In addition to the provided data, we will be using external datasets put together by participants in the Kaggle competition. You can download all of them [here](http://files.fast.ai/part2/lesson14/rossmann.tgz).

For completeness, the implementation used to put them together is included below.

In [3]:
def concat_csvs(dirname):
    path = f'{PATH}{dirname}'
    filenames=glob(f"{PATH}/*.csv")

    wrote_header = False
    with open(f"{path}.csv","w") as outputfile:
        for filename in filenames:
            name = filename.split(".")[0]
            with open(filename) as f:
                line = f.readline()
                if not wrote_header:
                    wrote_header = True
                    outputfile.write("file,"+line)
                for line in f:
                     outputfile.write(name + "," + line)
                outputfile.write("\n")

In [4]:
# concat_csvs('googletrend')
# concat_csvs('weather')

Feature Space:
* train: Training set provided by competition
* store: List of stores
* store_states: mapping of store to the German state they are in
* List of German state names
* googletrend: trend of certain google keywords over time, found by users to correlate well w/ given data
* weather: weather
* test: testing set

In [5]:
table_names = ['train', 'store', 'store_states', 'state_names', 
               'googletrend', 'weather', 'test']

We'll be using the popular data manipulation framework `pandas`. Among other things, pandas allows you to manipulate tables/data frames in python as one would in a database.

We're going to go ahead and load all of our csv's as dataframes into the list `tables`.

In [6]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

In [7]:
from IPython.display import HTML, display

We can use `head()` to get a quick look at the contents of each table:
* train: Contains store information on a daily basis, tracks things like sales, customers, whether that day was a holdiay, etc.
* store: general info about the store including competition, etc.
* store_states: maps store to state it is in
* state_names: Maps state abbreviations to names
* googletrend: trend data for particular week/state
* weather: weather conditions for each state
* test: Same as training table, w/o sales and customers


In [8]:
for t in tables: display(t.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


This is very representative of a typical industry dataset.

The following returns summarized aggregate information to each table accross each field.

In [9]:
for t in tables: display(t.describe())

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


,Store
count,1115.00000
mean,558.00000
std,322.01708
min,1.00000
25%,279.50000
50%,558.00000
75%,836.50000
max,1115.00000


,StateName,State
count,16,16
unique,16,16
top,Saarland,BB
freq,1,1


,trend
count,2072.000000
mean,63.814189
std,12.650246
min,0.000000
25%,55.000000
50%,64.000000
75%,72.000000
max,100.000000


,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,...,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,WindDirDegrees
count,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,...,15840.000000,15459.000000,15459.000000,15459.000000,15840.000000,15840.000000,3604.000000,15840.000000,14667.000000,15840.000000
mean,14.644129,10.388952,6.198990,8.587816,6.205808,3.626136,93.659596,74.282891,50.158586,1018.532197,...,1012.307955,24.057572,12.239796,7.025163,22.766604,11.972222,48.864317,0.831718,5.551306,175.896717
std,8.646012,7.379260,6.526391,6.244780,6.086768,6.128390,7.678530,13.486552,19.960216,7.788720,...,8.600585,8.976799,5.067944,4.980602,8.988618,5.872844,13.026954,2.513506,1.687710,101.588872
min,-11.000000,-13.000000,-15.000000,-14.000000,-15.000000,-73.000000,44.000000,30.000000,4.000000,976.000000,...,970.000000,0.000000,0.000000,0.000000,3.000000,2.000000,21.000000,0.000000,0.000000,-1.000000
25%,8.000000,4.000000,1.000000,4.000000,2.000000,-1.000000,90.750000,65.000000,34.000000,1014.000000,...,1007.000000,14.000000,10.000000,3.000000,16.000000,8.000000,39.000000,0.000000,5.000000,80.000000
50%,15.000000,11.000000,7.000000,9.000000,7.000000,4.000000,94.000000,76.000000,49.000000,1019.000000,...,1013.000000,31.000000,11.000000,7.000000,21.000000,11.000000,48.000000,0.000000,6.000000,202.000000
75%,21.000000,16.000000,11.000000,13.000000,11.000000,8.000000,100.000000,85.000000,66.000000,1024.000000,...,1018.000000,31.000000,14.000000,10.000000,27.000000,14.000000,55.000000,0.250000,7.000000,256.000000
max,39.000000,31.000000,24.000000,25.000000,20.000000,19.000000,100.000000,100.000000,100.000000,1043.000000,...,1038.000000,31.000000,31.000000,31.000000,101.000000,53.000000,111.000000,58.930000,8.000000,360.000000


,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000


## Data Cleaning / Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

In [10]:
train, store, store_states, state_names, googletrend, weather, test = tables

In [11]:
len(train),len(test)

(1017209, 41088)

We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [12]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [13]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

Join weather/state names.

In [14]:
weather = join_df(weather, state_names, "file", "StateName")

In pandas you can add new columns to a dataframe by simply defining it. We'll do this for googletrends by extracting dates and state names from the given data and adding those columns.

We're also going to replace all instances of state name 'NI' to match the usage in the rest of the data: 'HB,NI'. This is a good opportunity to highlight pandas indexing. We can use `.loc[rows, cols]` to select a list of rows and a list of columns from the dataframe. In this case, we're selecting rows w/ statename 'NI' by using a boolean list `googletrend.State=='NI'` and selecting "State".

In [15]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [16]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

The Google trends data has a special category for the whole of the Germany - we'll pull that out so we can use it explicitly.

In [17]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

Now we can outer join all of our data into a single dataframe. Recall that in outer joins everytime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has Null values for all right table fields. One way to check that all records are consistent and complete is to check for Null values post-join, as we do here.

*Aside*: Why note just do an inner join?
If you are assuming that all records are complete and match on the field you desire, an inner join will do the same thing as an outer join. However, in the event you are wrong or a mistake is made, an outer join followed by a null-check will catch it. (Comparing before/after # of rows for inner join is equivalent, but requires keeping track of before/after row #'s. Outer join is easier.)

In [18]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [19]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [20]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [21]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [22]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [23]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

Next we'll fill in missing values to avoid complications with `NA`'s. `NA` (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.

In [24]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

Next we'll extract features "CompetitionOpenSince" and "CompetitionDaysOpen". Note the use of `apply()` in mapping a function across dataframe values.

In [25]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

We'll replace some erroneous / outlying data.

In [26]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [27]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

Same process for Promo dates.

In [28]:
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [29]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [31]:
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [32]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

We'll be applying this to a subset of columns:

In [55]:
# Note: Run cells below this one first with df = train[columns], then with df=test[columns].
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

#df = train[columns]
df = test[columns]

In [56]:
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday
0,2015-09-17,1,1,False,0
1,2015-09-17,3,1,False,0
2,2015-09-17,7,1,False,0
3,2015-09-17,8,1,False,0
4,2015-09-17,9,1,False,0


Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call `add_elapsed('SchoolHoliday', 'After')`:
This will apply to each row with School Holiday:
* A applied to every row of the dataframe in order of store and date
* Will add to the dataframe the days since seeing a School Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [57]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We'll do this for two more fields.

In [58]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [59]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We're going to set the active index to Date.

In [60]:
df = df.set_index("Date")

Then set null values from elapsed field calculations to 0.

In [61]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [62]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

In [63]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [64]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

Next we want to drop the Store indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [65]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [66]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [67]:
df.reset_index(inplace=True)

Now we'll merge these values onto the df.

In [68]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [69]:
df.drop(columns,1,inplace=True)

In [70]:
df.head()

,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,13,0,0,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,12,0,0,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,11,0,0,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,10,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,9,0,0,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0


It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [71]:
df.to_feather(f'{PATH}df')

In [72]:
df = pd.read_feather(f'{PATH}df')

In [73]:
df["Date"] = pd.to_datetime(df.Date)

In [74]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw',
       'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [75]:
# Note: Run this cell first with 1st line, then with 2nd line. 

#joined = join_df(joined, df, ['Store', 'Date'])
joined_test = join_df(joined_test, df, ['Store', 'Date'])

The authors also removed all instances where the store had zero sale / was closed. We speculate that this may have cost them a higher standing in the competition. One reason this may be the case is that a little exploratory data analysis reveals that there are often periods where stores are closed, typically for refurbishment. Before and after these periods, there are naturally spikes in sales that one might expect. By ommitting this data from their training, the authors gave up the ability to leverage information about these periods to predict this otherwise volatile behavior.

In [54]:
joined = joined[joined.Sales!=0]

We'll back this up as well.

In [76]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [77]:
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

We now have our final set of engineered features.

While these steps were explicitly outlined in the paper, these are all fairly typical feature engineering steps for dealing with time series data and are practical in any similar setting.

## Create features

In [78]:
joined = pd.read_feather(f'{PATH}joined')
joined_test = pd.read_feather(f'{PATH}joined_test')

In [79]:
joined.head().T.head(40)

,0,1,2,3,4
index,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,1,1,1,1,1


Now that we've engineered all our features, we need to convert to input compatible with a neural network.

This includes converting categorical variables into contiguous integers or one-hot encodings, normalizing continuous features to standard normal, etc...

In [80]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

n = len(joined); n

844338

In [81]:
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

In [82]:
joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

In [83]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [84]:
apply_cats(joined_test, joined)

In [85]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

We're going to run on a sample.

In [ ]:
#idxs = get_cv_idxs(n, val_pct=50000/n)
#idxs = get_cv_idxs(n, val_pct=150000/n)
#joined_samp = joined.iloc[idxs].set_index("Date")
#samp_size = len(joined_samp); samp_size

To run on the full dataset, use this instead:

In [86]:
samp_size = n
joined_samp = joined.set_index("Date")
joined_test = joined_test.set_index("Date")

We can now process our data...

In [87]:
print(len(joined_samp))
joined_samp.head()

844338


,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,2015,7,31,False,24,0,c,a,...,24.0,11.0,1.0,85.0,83.0,57.0,0.0,1.0,1.0,5263
2015-07-31,2,5,2015,7,31,False,24,25,a,a,...,14.0,11.0,4.0,80.0,83.0,67.0,0.0,1.0,1.0,6064
2015-07-31,3,5,2015,7,31,False,24,25,a,a,...,14.0,5.0,2.0,86.0,83.0,57.0,0.0,1.0,1.0,8314
2015-07-31,4,5,2015,7,31,False,24,0,c,c,...,23.0,16.0,6.0,74.0,83.0,67.0,0.0,1.0,1.0,13995
2015-07-31,5,5,2015,7,31,False,3,0,a,a,...,14.0,11.0,4.0,82.0,83.0,57.0,0.0,1.0,1.0,4822


In [88]:
print(len(joined_test))
joined_test.head()

41088


,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday,Sales,Id
Date,,,,,,,,,,,,,,,,,,,,,
2015-09-17,1,4,2015,9,17,False,24,0,c,a,...,14.0,6.0,69.0,67.0,0.0,0.0,1.0,0.0,0,1
2015-09-17,3,4,2015,9,17,False,24,25,a,a,...,26.0,6.0,68.0,67.0,0.0,0.0,1.0,0.0,0,2
2015-09-17,7,4,2015,9,17,False,24,0,a,c,...,14.0,5.0,59.0,67.0,0.0,0.0,1.0,0.0,0,3
2015-09-17,8,4,2015,9,17,False,11,0,a,a,...,14.0,5.0,59.0,67.0,0.0,0.0,1.0,0.0,0,4
2015-09-17,9,4,2015,9,17,False,24,0,a,c,...,26.0,6.0,68.0,67.0,0.0,0.0,1.0,0.0,0,5


In [89]:
val_idxs = np.flatnonzero(
    (joined_samp.index<=datetime.datetime(2014,9,17)) & (joined_samp.index>=datetime.datetime(2014,8,1)))

val_idxs = list(val_idxs)
print(len(val_idxs),type(val_idxs))

38399 <class 'list'>


### Part (2) - Construct and train a learner for pre-processed data using only my own functions and classes 

Note: Loss Function specified in the Rossmann kaggle competition is the root-mean-square-percentage-error (RMSPE):

$ RMSPE(\hat{y},y) = \sqrt{ \frac{1}{N} \sum_{i=1}^N \left(\frac{y_i - \hat{y}_i}{y_i}\right)^2 } $

where $N$ is the number of datapoints, $y_i$ is the ith sales value, and $\hat{y}_i$ is its predicted value. In my code I will follow the same approach used in the Fastai notebook, which is to instead train the network by minimizing the Mean-Squared-Error of the log of the sales-data:

$ loss = \frac{1}{N} \sum_{i=1}^N \left(\log(y_i) - \log(\hat{y}_i)\right)^2 $

(I will also, of course, compute the RMSPE loss for the predicted outputs from the network.)

In [90]:
# Split into train and val DataFrames
train_df, val_df = SD.SplitDataFrameTrainVal(joined_samp,val_idxs=val_idxs)
print(len(train_df))
print(len(val_df))
print('')

805939
38399



In [91]:
test_df = joined_test.drop(['Sales','Id'],axis=1)
test_df.head()

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,Promo,SchoolHoliday
Date,,,,,,,,,,,,,,,,,,,,,
2015-09-17,1,4,2015,9,17,False,24,0,c,a,...,88.0,29.0,14.0,6.0,69.0,67.0,0.0,0.0,1.0,0.0
2015-09-17,3,4,2015,9,17,False,24,25,a,a,...,58.0,37.0,26.0,6.0,68.0,67.0,0.0,0.0,1.0,0.0
2015-09-17,7,4,2015,9,17,False,24,0,a,c,...,61.0,29.0,14.0,5.0,59.0,67.0,0.0,0.0,1.0,0.0
2015-09-17,8,4,2015,9,17,False,11,0,a,a,...,61.0,29.0,14.0,5.0,59.0,67.0,0.0,0.0,1.0,0.0
2015-09-17,9,4,2015,9,17,False,24,0,a,c,...,58.0,37.0,26.0,6.0,68.0,67.0,0.0,0.0,1.0,0.0


In [92]:
# Construct the data object
cat_vars = cat_vars
cont_vars = contin_vars + ['Sales']
output_var = 'Sales'
bs = 1024

# train_ds
scale_cont = 'by_df'
xcat_df, xcont_df, y, scaling_values, category_labels = \
SD.ProcessDataFrame(train_df, cat_vars, cont_vars, output_var,'by_df')
ylog = list(np.log(y))
train_ds = SD.StructuredDataset(xcat_df,xcont_df,ylog,'cont')

# val_ds
scale_cont = scaling_values
xcat_df, xcont_df, y, scaling_values, category_labels = \
SD.ProcessDataFrame(val_df, cat_vars, cont_vars, output_var, scale_cont, category_labels=category_labels)
ylog = list(np.log(y))
val_ds = SD.StructuredDataset(xcat_df,xcont_df,ylog,'cont')

# test_ds
scale_cont = scaling_values
cont_vars.remove('Sales')
xcat_df, xcont_df, y, scaling_values, category_labels = \
SD.ProcessDataFrame(test_df, cat_vars, cont_vars, None, scale_cont, category_labels=category_labels)
test_ds = SD.StructuredDataset(xcat_df,xcont_df,y,'cont')

# structured data object
data = SD.StructuredDataObj(train_ds, val_ds, category_labels, scaling_values, bs, test_ds = test_ds)    

/home/paperspace/fastai/courses/dl1/StructuredData.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for var in cat_vars: df[var] = df[var].astype('category')
/home/paperspace/fastai/courses/dl1/StructuredData.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for var in cont_vars: df[var] = df[var].astype('float32')


In [112]:
# construct model 
fc_layer_sizes = [1000,500,1]
emb_sizes = 'default'
output_range = data.train_ds.y_range()
output_range2 = data.val_ds.y_range()
print('train data output_range = ',output_range)
print('validation data output_range =',output_range2)
output_range = [0,output_range[1]*1.2]
print('maximum output range of model = ', output_range)
#dropout_levels = (0.04,0,[0,0.001,0.01])
dropout_levels = (0,0,[0,0.2,0.2])
#dropout_levels = (0,0,[0,0.5,0.5])
use_bn = True
model = SD.StructuredDataNet.from_dataobj(data, fc_layer_sizes, emb_sizes, output_range, dropout_levels, use_bn)
model

train data output_range =  [3.8286414, 10.634677]
validation data output_range = [6.650279, 10.431554]
maximum output range of model =  [0, 12.76161231994629]


StructuredDataNet(
  (embeddings): ModuleList(
    (0): Embedding(1116, 50)
    (1): Embedding(8, 4)
    (2): Embedding(4, 2)
    (3): Embedding(13, 7)
    (4): Embedding(32, 16)
    (5): Embedding(3, 2)
    (6): Embedding(26, 13)
    (7): Embedding(27, 14)
    (8): Embedding(5, 3)
    (9): Embedding(4, 2)
    (10): Embedding(4, 2)
    (11): Embedding(24, 12)
    (12): Embedding(9, 5)
    (13): Embedding(13, 7)
    (14): Embedding(53, 27)
    (15): Embedding(22, 11)
    (16): Embedding(7, 4)
    (17): Embedding(7, 4)
    (18): Embedding(4, 2)
    (19): Embedding(4, 2)
    (20): Embedding(9, 5)
    (21): Embedding(9, 5)
  )
  (emb_drop): Dropout(p=0)
  (cont_drop): Dropout(p=0)
  (fc_net): FullyConnectedNet(
    (dropoutlayers): ModuleList(
      (0): Dropout(p=0)
      (1): Dropout(p=0.2)
      (2): Dropout(p=0.2)
    )
    (batchnormlayers): ModuleList(
      (0): BatchNorm1d(215, eps=1e-05, momentum=0.1, affine=True)
      (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)


In [113]:
# define optimizer and loss function
optimizer = optim.Adam(model.parameters())
loss_func = nn.MSELoss()

# define metrics
def Exp_MSPE(y_pred,y):
    pct_var = (y.exp() - y_pred.exp())/y.exp()
    return (pct_var**2).mean()

# construct learner
learner = Gen.Learner(PATH,data,model,optimizer,loss_func)

In [114]:
learner.fit(lr=0.003,num_epochs=3,save_name = 'rossmann',metrics=[Exp_MSPE])
print('')

epoch   train_loss  val_loss    metrics     

0       0.01882     0.01844     0.01820       epoch run time: 0 min, 24.98 sec
1       0.01538     0.01684     0.01709       epoch run time: 0 min, 24.79 sec
2       0.01309     0.01685     0.01837       epoch run time: 0 min, 25.56 sec



In [115]:
learner.fit(lr=0.003,num_epochs=3,save_name = 'rossmann',metrics=[Exp_MSPE])
print('')

epoch   train_loss  val_loss    metrics     

0       0.01227     0.01805     0.01534       epoch run time: 0 min, 24.13 sec
1       0.01104     0.01669     0.01444       epoch run time: 0 min, 24.49 sec
2       0.00989     0.01502     0.01332       epoch run time: 0 min, 23.49 sec



In [116]:
learner.load('rossmann')
learner.fit(lr=0.003, num_cycles=3, base_cycle_length=1, save_name = 'rossmann', metrics=[Exp_MSPE])
print('')

epoch   train_loss  val_loss    metrics     

0       0.00722     0.01234     0.01196       epoch run time: 0 min, 23.79 sec
1       0.00720     0.01239     0.01185       epoch run time: 0 min, 24.04 sec
2       0.00684     0.01207     0.01151       epoch run time: 0 min, 24.50 sec



In [117]:
learner.load('rossmann')
learner.fit(lr=0.003, num_cycles=3, base_cycle_length=2, save_name = 'rossmann', metrics=[Exp_MSPE])
print('')

epoch   train_loss  val_loss    metrics     

0       0.00745     0.01315     0.01212       epoch run time: 0 min, 24.15 sec
1       0.00634     0.01145     0.01113       epoch run time: 0 min, 23.83 sec
2       0.00714     0.01212     0.01239       epoch run time: 0 min, 24.22 sec
3       0.00612     0.01131     0.01126       epoch run time: 0 min, 24.01 sec
4       0.00735     0.01117     0.01109       epoch run time: 0 min, 24.40 sec
5       0.00576     0.01121     0.01080       epoch run time: 0 min, 23.87 sec



In [119]:
learner.load('rossmann')
learner.fit(lr=0.003, num_cycles=3, base_cycle_length=4, save_name = 'rossmann2', metrics=[Exp_MSPE])
print('')

epoch   train_loss  val_loss    metrics     

0       0.00664     0.01219     0.01135       epoch run time: 0 min, 24.31 sec
1       0.00601     0.01101     0.01063       epoch run time: 0 min, 24.15 sec
2       0.00538     0.01103     0.01061       epoch run time: 0 min, 24.07 sec
3       0.00528     0.01082     0.01052       epoch run time: 0 min, 24.90 sec
4       0.00687     0.01199     0.01197       epoch run time: 0 min, 23.52 sec
5       0.00604     0.01168     0.01138       epoch run time: 0 min, 24.04 sec
6       0.00505     0.01075     0.01037       epoch run time: 0 min, 23.88 sec
7       0.00497     0.01089     0.01042       epoch run time: 0 min, 24.46 sec
8       0.00625     0.01163     0.01201       epoch run time: 0 min, 24.48 sec
9       0.00522     0.01077     0.01056       epoch run time: 0 min, 23.68 sec
10      0.00512     0.01117     0.01071       epoch run time: 0 min, 24.59 sec
11      0.00471     0.01060     0.01028       epoch run time: 0 min, 24.32 sec



In [120]:
# Best RMSPE on Validation Set (from epoch 11)
np.sqrt(0.01028)

0.1013903348450926

In [121]:
learner.load('rossmann2')

# Check predictions on validation set are same as best saved performance
y_pred = learner.predict()
print(y_pred)
y = learner.data.val_ds.y
print(y)
y = Variable(torch.Tensor(y))
y_pred = Variable(torch.Tensor(y_pred))
loss = loss_func(y_pred,y)
print(loss)

[ 8.37233  8.7272   8.93424  9.10514  8.57243  8.59799  9.07086  8.69407  8.81908  8.65073  9.01146  8.98311
  8.65083  8.75819  9.02351  8.86038  8.83788  8.81958  8.58372  8.59858 ...  8.91719  9.16072  9.19597
  8.50314  8.83893  9.18377  8.72842  9.19392  8.74111  9.3031   8.70988  8.69871  8.61851  8.82409  8.51505
  8.56215  9.21207  8.93957 10.02994  8.93809]
[ 8.38549  8.77478  8.99144  9.05882  8.64559  8.57508  9.06114  8.70583  8.72453  8.65067  9.0281   8.93722
  8.57508  8.77771  8.97563  8.86672  8.8835   8.80897  8.64997  8.63355 ...  8.87654  9.19523  9.10064
  8.52337  8.8694   9.18009  8.74241  9.10487  8.74799  9.32643  8.53464  8.55526  8.57244  8.74018  8.5713
  8.55622  9.21612  8.93827 10.0357   9.05719]
Variable containing:
1.00000e-02 *
  1.0604
[torch.FloatTensor of size 1]



In [122]:
# Make predictions for test set and save to csv file
sales_pred = np.exp(learner.predict(test=True))
Id = list(range(1,len(sales_pred)+1))

rossmann_df = pd.DataFrame({"Id":Id,"Sales":sales_pred})
rossmann_df.to_csv(PATH+'rossmann_preds',index=False)